# Experiment 01: Band configurations

In this experiment we will study how different band configurations affect the training process and end results.

Configurations:

- E01-8: All 8 MS WorldView-2 bands is used.
- E01-6: The 6 MS WorldView-2 bands that overlap the PAN band (RGB+Y+RedEdge+NIR1)
- E01-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E01-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set

In [ ]:
import tensorflow as tf

from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *

# Check GPUs:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
# Load metadata dataframe from repository root
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)

# Paths
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e01'

# Subset by sensor and area
SENSORS = ['WV02', 'GE01']
AREAS = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS, sensors=SENSORS)
print('Sensors:', SENSORS)
print('Areas:', AREAS)

# Count images in partitions
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

# Set tile dimensions
SR_FACTOR = 4
MS_SIZE_TRAIN = 32
PAN_SIZE_TRAIN = MS_SIZE_TRAIN * SR_FACTOR
print('MS (LR) tile size:', MS_SIZE_TRAIN, 'x', MS_SIZE_TRAIN,  '(training set)')
print('PAN (HR) tile size:', PAN_SIZE_TRAIN, 'x', PAN_SIZE_TRAIN,  '(training set)')

MS_SIZE_VAL = 128
PAN_SIZE_VAL = MS_SIZE_VAL * SR_FACTOR
print('MS (LR) tile size:', MS_SIZE_VAL, 'x', MS_SIZE_VAL,  '(validation set)')
print('PAN (HR) tile size:', PAN_SIZE_VAL, 'x', PAN_SIZE_VAL,  '(validation set)')

# Band (channel) configurations:
PAN_BANDS = 1


CLOUD_SEA_REMOVAL = True



BATCH_SIZE_TRAIN = 16
BATCH_SIZE_VAL = 8

SHAVE_WIDTH = 4
MATLAB_PATH = 'modules/matlab'

In [ ]:
wv02 = get_sensor_band_config('WV02', meta)
ge01 = get_sensor_band_config('GE01', meta)

In [ ]:
wv02